<a href="https://colab.research.google.com/github/Rishabh9559/Vendor_Performance_Data_Analytics/blob/main/ingestion_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import packages
import pandas as pd
import os
from sqlalchemy import create_engine
import logging
import time

In [6]:
# logs configuration
logging.basicConfig(
    filename = 'logs/ingestion.log',
    level = logging.DEBUG,
    format = '%(asctime)s - %(levelname)s - %(message)s',
    filemode = 'a'

)

In [7]:
# sqlite engine creation

engine = create_engine(
    'sqlite:///vender_performance.db')

In [8]:
# ingest the dataframe into the database
# using chunck size to prevent the RAM crash
def ingest_db(df, table_name, engine, chunck_size):
    try:
        df.to_sql(table_name, con = engine, if_exists = 'replace', index = False, chunksize = 10000)

    except Exception as e:
        logging.error(f'Error {e} occurred while ingesting data into database')

In [9]:
# loda raw data

''' This function loads the raw data from the data folder and ingests it into the database. It also logs the time taken for the ingestion process and any errors that occur during the process.'''

def load_data():
    try:
        start = time.time()
        for file in os.listdir('data'):
            if '.csv' in file:
                df = pd.read_csv('data/'+file)
                print(f'Database {file} and its {df.shape}')
                logging.info(f'Ingesting {file} into database')
                # Call ingest_db with a specified chunksize (e.g., 10000 rows at a time)
                ingest_db(df, file[:-4], engine, chunck_size=10000)
        end = time.time()
        total_time = (end - start)/60
        logging.info(f'---------------Data Ingestion Completed in {total_time} minutes-----------------')

    except Exception as e:
        logging.error(f'Error {e} occurred during data ingestion')


In [10]:
# calling the load_data function
load_data()

Database purchase_prices.csv and its (12261, 9)
Database vendor_invoice.csv and its (5543, 10)
Database end_inventory.csv and its (224489, 9)
Database sales.csv and its (12825363, 14)
Database purchases.csv and its (2372474, 16)
Database begin_inventory.csv and its (206529, 9)
